In [156]:
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import osmnx as ox
import pandas as pd
import pickle
from pathlib import Path
import os
from tqdm import tqdm
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import requests
import zipfile


EPSILON = 1e-3

In [157]:
class Agent:
    
    def __init__(self, i, dow, city, alpha=0.5):

        ''' 
        Initialize an Agent instance
        Parameters:
        - i (int): Agent identifier.
        - dow (float): Endowment value.
        - city (City): Reference to the City instance.
        - alpha (float): Weighting factor for transit access vs. community value.
        '''
        self.i = i
        self.dow = dow
        self.city = city 
        self.alpha = alpha

        self.weights = np.ones(len(self.city.centroidDict))
        self.probabilities = np.ones(len(self.city.centroidDict)) # Probability to go to each centroid
        self.tot_probabilities = 0.0
        self.avg_probabilities = None
        self.u = None

        self.reset()
        
    # Create hash identifier
    def __hash__(self):
        return hash(self.i)

    def __eq__(self, other): 
        return self.i == other.i

    '''
    - Assign starting centroid
    - Reset centroid weights/probabilities
    '''
    def reset(self):
        # Normalize probabilities
        self.probabilities[:] = [a/len(self.probabilities) for a in self.probabilities]
        
        self.tot_probabilities = np.sum(self.probabilities)
        
        # Current node - Initialize starting position at random node (based on weights)
        self.u = np.random.choice(list(self.city.centroidDict.keys()), p=self.probabilities) 
        
        # Adds self to node
        self.city.centroidDict[self.u]['inh'].add(self)

# ACTION METHOD
    def act(self): 
        # Leave node
        self.city.centroidDict[self.u]['inh'].remove(self) 
    
        # Choose another node
        self.u = np.random.choice(list(self.city.centroidDict.keys()), p=self.probabilities) 
    
        # Join node
        self.city.centroidDict[self.u]['inh'].add(self) 
        
# LEARN METHOD
    def learn(self):
        for ID, _ in self.city.centroidDict.items():
            self.weights[ID] *= (1 - EPSILON * self.cost(ID)) # Weighted based on COST
        
        self.probabilities = np.array(self.weights / np.sum(self.weights)) # Normalize
        
        for a in self.probabilities:
            self.tot_probabilities += a # used for averaging purposes

# COST FUNCTION
    def cost(self, ID):
        
        # AFFORDABILITY SCORE
        # 1 if self.dow >= node.dow_thr; else 0
        aff = int(self.dow >= self.city.centroidDict[ID]['dow_thr'])
        
        # UPKEEP SCORE
        # 1 if upkeep == True; else 0
        upk = int(self.city.centroidDict[ID]['upk'])
        
        # BELTLINE SCORE
        # 1 if in beltline; else 0
        beltline = int(self.city.centroidDict[ID]['beltline'])
    
        # DISTANCE SCORE
        loc = np.exp(- (1 - self.alpha) * self.city.centroid_distances[self.u, ID])
        
        # COMMUNITY SCORE
        # Difference between node 'cmt' value and self.dow.
        cmt = np.exp(- self.alpha * np.abs(self.dow - self.city.centroidDict[ID]['cmt']))
    
        # COST FUNCTION
        c = 1 - aff * loc * beltline * upk * cmt
        
        return c

In [158]:
class City:

    # CONSTRUCTOR
    def __init__(self, centroids, g, rho=2): #default rho (house capacity) == 2
        '''
        Initialize a City instance.
        '''
        self.rho = rho #house capacity
        self.centroid = centroids #centroids list
        self.g = g #OSMnx map
        
        # Create a dictionary with each centroid's attributes
        self.centroidDict = {} 
        for ID, (lat, lon, name, beltline) in enumerate(self.centroid): #Iterate through each centroid
            self.centroidDict[ID] = {
                'lat': lat, # Latitude
                'lon': lon, # Longitude
                'name': name, # Housing name (region)
                'beltline': beltline,  # Is it in the Beltline?
                
                'inh': set(),  # Set containing all Agent inhabitants
                'dow_thr': 0.0,  # Endowment threshold initialized to 0
                'upk': False,  # Upkeep score
                'cmt': 0.0,  # Community score
                'pop_hist': [], # Population history
                'cmt_hist': [],  # Community history
                
                'node': ox.nearest_nodes(self.g, lon, lat)
            }
        
        # Initialize dictionary of distances between centroids
        self.centroid_distances = self.compute_centroid_distances()
        
        
    # Distances Between Centroids - helper function
    def compute_centroid_distances(self):
        n = len(self.centroidDict)
        distance_matrix = np.zeros((n, n))
        
        for i in tqdm(range(n), desc="Computing centroid distances"):
            source_node = self.centroidDict[i]['node']
            lengths = nx.single_source_dijkstra_path_length(self.g, source_node, weight='length')
            for j in range(n):
                target_node = self.centroidDict[j]['node']
                distance = lengths.get(target_node, np.inf)
                distance_matrix[i][j] = distance
                    
        max_distance = distance_matrix.max()
        if max_distance > 0:
            distance_matrix /= max_distance

        return distance_matrix


    def set_agts(self, agts):
        self.agts = agts #list of agents
        self.agt_dows = np.array([a.dow for a in self.agts]) #array of agent endowments

    # Update each node
    def update(self):   
        for ID, data in self.centroidDict.items(): # For each centroid

            pop = len(data['inh']) # Inhabitants
            inhabitant_dows = [a.dow for a in data['inh']]  # Array of endowments of node's inhabitants
            
            # COMMUNITY SCORE (average endowment)
            cmt = 0.0
            if pop > 0:
                distances = self.centroid_distances[ID, [agent.u for agent in data['inh']]]
                weights = (1 - distances) ** 2
                
                if np.sum(weights) > 0:
                    cmt = np.average(inhabitant_dows, weights=weights)
                data['cmt'] = cmt
            
            # UPKEEP SCORE
            # ENDOWMENT THRESHOLD
            if pop > 0: # If inhabited
                if pop < self.rho:
                    data['dow_thr'] = 0.0
                else:
                    data['dow_thr'] = np.partition(inhabitant_dows, -self.rho)[-self.rho] # Lowest endowment value if Population = Rho
                data['upk'] = True
                
            else: # If uninhabited
                data['dow_thr'] = 0.0
                data['upk'] = False

            # Update population history
            data['pop_hist'].append(pop)
            # Update Community history (average endowment)
            data['cmt_hist'].append(cmt)
        
        
    # =============
    # PLOTTING CODE
    # =============
    def plot(self, cmap='YlOrRd', figkey='city', graph=None):
        fig, ax = plt.subplots(figsize=(10, 10))
    
        if graph:
            ox.plot_graph(graph, ax=ax, node_color='black', node_size=10, edge_color='gray', edge_linewidth=1, show=False, close=False)
    
        # Prepare agent data
        agent_lats = [self.centroidDict[agent.u]['lat'] for agent in self.agts]
        agent_lons = [self.centroidDict[agent.u]['lon'] for agent in self.agts]
        agent_wealths = [agent.dow for agent in self.agts]
    
        # Population density heatmap
        heatmap, xedges, yedges = np.histogram2d(agent_lons, agent_lats, bins=30)
        extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
        ax.imshow(heatmap.T, extent=extent, origin='lower', cmap=cmap, alpha=0.5)
    
        # Plot agents with wealth-based marker sizes
        norm = Normalize(vmin=min(agent_wealths), vmax=max(agent_wealths))
        marker_sizes = [50 + 150 * norm(w) for w in agent_wealths]
        sc = ax.scatter(agent_lons, agent_lats, c=agent_wealths, s=marker_sizes, cmap='coolwarm', alpha=0.7, edgecolor='red')
    
        # Plot centroids locations (this comes after the graph to make sure they are visible on top)
        for ID, house in self.centroidDict.items():
            lat, lon = house['lat'], house['lon']
            color = 'yellow' if house['beltline'] else 'white'
            ax.scatter(lon, lat, color=color, s=100, alpha=0.7, edgecolor='black')
            
            # Display inhabitant populations at each node:
            inhabitants = len(house['inh'])
            ax.text(lon, lat, str(inhabitants), fontsize=9, ha='center', va='center', color='black')

    
        # Add color bar for wealth
        cbar = plt.colorbar(sc, ax=ax, orientation='vertical', label='Wealth (dow)')
    
        # Labels and title
        ax.set_title(f"City Visualization: {figkey}")
        ax.set_xlabel("Longitude")
        ax.set_ylabel("Latitude")
    
        # Legend
        ax.scatter([], [], c='yellow', s=100, label='Beltline Housing')
        ax.scatter([], [], c='white', s=100, label='Non-Beltline Housing')
        ax.scatter([], [], c='red', s=100, label='Agents')
        ax.legend(loc='upper right')
    
        plt.tight_layout()
        plt.savefig(f'./figures/{figkey}.pdf', format='pdf', bbox_inches='tight')
        plt.close()

In [159]:
def download_and_extract_file(url, filename):
    # Use a 'data' subfolder in the current working directory
    cwd = Path.cwd()
    data_dir = cwd / "data"
    data_dir.mkdir(exist_ok=True)  # Create the 'data' directory if it doesn't exist
    file_path = data_dir / filename

    # Create extraction subfolder name (remove .zip extension)
    extract_folder_name = filename.rsplit('.', 1)[0]
    extract_path = data_dir / extract_folder_name
    
    # Check if file already exists
    if file_path.exists():
        print(f"{filename} already exists in {data_dir}. Skipping download.")
    else:
        # Make the request
        print(f"Downloading {filename} to {data_dir}...")
        response = requests.get(url, stream=True)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Get the total file size
            total_size = int(response.headers.get('content-length', 0))

            # Open the file and use tqdm for the progress bar
            with file_path.open('wb') as file, tqdm(
                desc=filename,
                total=total_size,
                unit='iB',
                unit_scale=True,
                unit_divisor=1024,
            ) as progress_bar:
                for data in response.iter_content(chunk_size=1024):
                    size = file.write(data)
                    progress_bar.update(size)
            print(f"Successfully downloaded {filename}")
        else:
            print(f"Failed to download {filename}. Status code: {response.status_code}")
            return

    # Extract the ZIP file
    print(f"Extracting {filename} to {extract_path}...")
    extract_path.mkdir(exist_ok=True)  # Create the extraction folder if it doesn't exist
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        # Get the total number of files in the ZIP
        total_files = len(zip_ref.infolist())
        
        # Use tqdm for the extraction progress bar
        for file in tqdm(zip_ref.infolist(), desc="Extracting", total=total_files):
            zip_ref.extract(file, extract_path)
    
    print(f"Successfully extracted {filename} to {extract_path}")

# URL of the file to download
url = "https://www2.census.gov/geo/tiger/TIGER2022/ZCTA520/tl_2022_us_zcta520.zip"

# Filename to save as
filename = "tl_2022_us_zcta520.zip"

# Call the function to download and extract the file
download_and_extract_file(url, filename)

tl_2022_us_zcta520.zip already exists in c:\Users\kmmat\OneDrive\Desktop\VIP\24Fa-MPONC\modeling_processes_of_neighborhood_change_new\data. Skipping download.
Extracting tl_2022_us_zcta520.zip to c:\Users\kmmat\OneDrive\Desktop\VIP\24Fa-MPONC\modeling_processes_of_neighborhood_change_new\data\tl_2022_us_zcta520...


Extracting: 100%|██████████| 7/7 [00:04<00:00,  1.64it/s]

Successfully extracted tl_2022_us_zcta520.zip to c:\Users\kmmat\OneDrive\Desktop\VIP\24Fa-MPONC\modeling_processes_of_neighborhood_change_new\data\tl_2022_us_zcta520


In [160]:
cwd = Path.cwd()
print(cwd)
figures_folder = Path(cwd / "figures")
if not os.path.isdir(figures_folder):
    os.makedirs(figures_folder)


# Centroids
'''centroids = [
    (33.7501, 84.3885, 'RDA/Cascade', True),
    (33.7501, 84.3885, 'Pittsburgh/Peoplestown', True),
    (33.7501, 84.3885, 'Boulevard Crossing', True),
    (33.73586185,-84.3709322239104, 'Memorial Drive/Glenwood/Grant Park', True),
    (33.7680818,-84.36505111969021, 'Freedom Parkway/Fourth Ward Park', True),
    (33.7501, 84.3885, 'Virginia Highlands/Ansley/Piedmont Park', True),
    (0.0, 0.0, 'Peachtree/Collier', True),
    (0.0, 0.0, 'Upper Westside/Northside', True),
    (0.0, 0.0, 'Simpson/Hollowell', True),
    (33.779241999999996,-84.43839416079305, 'Upper Marietta/Westside Park', True)'''
centroids = [
    (33.6534427,-84.4493725, 'College Park', False),
    (33.7034405,-84.3143682, 'Gresham Park', False),
    (33.6795531,-84.4393724, 'East Point', False),
    (33.9242688,-84.3785379, 'Sandy Springs', True),
    (33.7737582,-84.296069, 'Decatur', False),
    (33.78904775,-84.37188620639064, 'Virginia Highlands/Ansley/Piedmont Park', True),
    (33.73586185,-84.3709322239104, 'Memorial Drive/Glenwood/Grant Park', True),
    (33.7680818,-84.36505111969021, 'Freedom Parkway/Fourth Ward Park', True),
    (33.779241999999996,-84.43839416079305, 'Upper Marietta/Westside Park', True)
]

# (GRAPH APPROACH - automate creation of nodes) 
# Getting nodes with OSMnx
load_g = False
if not load_g:
    '''gdf = gpd.read_file(cwd / Path('data/tl_2022_us_zcta520/tl_2022_us_zcta520.shp'))
    gdf = gdf[gdf['ZCTA5CE20'] == '11206']
    shape = gdf.iloc[0].geometry'''
    #IF LOADING SPECIFIC SHAPEFILE(S)
   
    places = [
    'Atlanta',
    'College Park, Georgia',
    'Gresham park, panthersville, georgia',
    'East Point, Georgia',
    'Sandy Springs, Georgia',
    'Decatur, DeKalb County, Georgia, 30030, United States',
    'Piedmont park, Georgia',
    'Westside Park, Atlanta',
    'Fourth Ward Park, Atlanta',
    'Westside Park, Atlanta'
    ]
    g = ox.graph_from_place(places, network_type='drive', simplify=True)
    #Roadmap of Atlanta
    g = g.subgraph(max(nx.strongly_connected_components(g), key=len)).copy()
    #Ensures all nodes are connected
    g = nx.convert_node_labels_to_integers(g)
    #Converts nodes to integers
   
    with open(Path(cwd / 'data/tl_2022_us_zcta520/atlanta.pkl'), 'wb') as file:
        pickle.dump(g, file)
else:
    with open(Path(cwd / 'data/tl_2022_us_zcta520/atlanta.pkl'), 'rb') as file:
        g = pickle.load(file)


# ====================================
# SIMULATION PRE-DETERMINED PARAMETERS
# ====================================
rho_l = [2, 4] #1, 2, 4, 8 (for each iteration) rho-house capacity
alpha_l = [0.25] #0.25, 0.75 (for each iteration) lambda - transit access vs. community value
t_max_l = [20000] #5000, 10000, 15000, 20000 (for each iteration) timesteps
tau = 0.5 # inequality factor in Lorentz curve
num_agents = 50

# RUN SIMULATION?
run_experiments = True

# PLOT SIMULATION?
plot_cities = True

cty_key = 'Atlanta'


# ===============
# SIMULATION CODE
# ===============

if run_experiments:
    for rho in rho_l:
        for alpha in alpha_l:

            np.random.seed(0)

            city = City(centroids, g, rho=rho)
            agt_dows = np.diff([1 - (1 - x) ** tau for x in np.linspace(0, 1, num_agents + 1)]) 
            agts = [Agent(i, dow, city, alpha=alpha) for i, dow in enumerate(agt_dows)]

            city.set_agts(agts)
            city.update()

            for t in range(max(t_max_l)):
                print('t: {0}'.format(t))
                for a in agts:
                    a.act()
                city.update()
                for a in agts:
                    a.learn()
                
                if t + 1 in t_max_l:

                    for a in city.agts:
                        a.avg_probabilities = a.tot_probabilities / (t + 1)

                    with open(Path(cwd / 'data/{0}_{1}_{2}_{3}.pkl'.format(cty_key, rho, alpha, t + 1)), 'wb') as file:
                        pickle.dump(city, file)

if plot_cities:
    for rho in rho_l:
        for alpha in alpha_l:
            for t_max in t_max_l:
                with open(Path(cwd / 'data/{0}_{1}_{2}_{3}.pkl'.format(cty_key, rho, alpha, t_max)), 'rb') as file:
                    city = pickle.load(file)
                cmap = 'YlOrRd'
                figkey = '{0}_{1}_{2}_{3}'.format(cty_key, rho, alpha, t_max)
                city.plot(cmap=cmap, figkey=figkey, graph=g)

c:\Users\kmmat\OneDrive\Desktop\VIP\24Fa-MPONC\modeling_processes_of_neighborhood_change_new


Computing centroid distances: 100%|██████████| 9/9 [00:00<00:00, 10.99it/s]


t: 0
t: 1
t: 2
t: 3
t: 4
t: 5
t: 6
t: 7
t: 8
t: 9
t: 10
t: 11
t: 12
t: 13
t: 14
t: 15
t: 16
t: 17
t: 18
t: 19
t: 20
t: 21
t: 22
t: 23
t: 24
t: 25
t: 26
t: 27
t: 28
t: 29
t: 30
t: 31
t: 32
t: 33
t: 34
t: 35
t: 36
t: 37
t: 38
t: 39
t: 40
t: 41
t: 42
t: 43
t: 44
t: 45
t: 46
t: 47
t: 48
t: 49
t: 50
t: 51
t: 52
t: 53
t: 54
t: 55
t: 56
t: 57
t: 58
t: 59
t: 60
t: 61
t: 62
t: 63
t: 64
t: 65
t: 66
t: 67
t: 68
t: 69
t: 70
t: 71
t: 72
t: 73
t: 74
t: 75
t: 76
t: 77
t: 78
t: 79
t: 80
t: 81
t: 82
t: 83
t: 84
t: 85
t: 86
t: 87
t: 88
t: 89
t: 90
t: 91
t: 92
t: 93
t: 94
t: 95
t: 96
t: 97
t: 98
t: 99
t: 100
t: 101
t: 102
t: 103
t: 104
t: 105
t: 106
t: 107
t: 108
t: 109
t: 110
t: 111
t: 112
t: 113
t: 114
t: 115
t: 116
t: 117
t: 118
t: 119
t: 120
t: 121
t: 122
t: 123
t: 124
t: 125
t: 126
t: 127
t: 128
t: 129
t: 130
t: 131
t: 132
t: 133
t: 134
t: 135
t: 136
t: 137
t: 138
t: 139
t: 140
t: 141
t: 142
t: 143
t: 144
t: 145
t: 146
t: 147
t: 148
t: 149
t: 150
t: 151
t: 152
t: 153
t: 154
t: 155
t: 156
t: 157
t: 1

Computing centroid distances: 100%|██████████| 9/9 [00:00<00:00, 11.20it/s]


t: 0
t: 1
t: 2
t: 3
t: 4
t: 5
t: 6
t: 7
t: 8
t: 9
t: 10
t: 11
t: 12
t: 13
t: 14
t: 15
t: 16
t: 17
t: 18
t: 19
t: 20
t: 21
t: 22
t: 23
t: 24
t: 25
t: 26
t: 27
t: 28
t: 29
t: 30
t: 31
t: 32
t: 33
t: 34
t: 35
t: 36
t: 37
t: 38
t: 39
t: 40
t: 41
t: 42
t: 43
t: 44
t: 45
t: 46
t: 47
t: 48
t: 49
t: 50
t: 51
t: 52
t: 53
t: 54
t: 55
t: 56
t: 57
t: 58
t: 59
t: 60
t: 61
t: 62
t: 63
t: 64
t: 65
t: 66
t: 67
t: 68
t: 69
t: 70
t: 71
t: 72
t: 73
t: 74
t: 75
t: 76
t: 77
t: 78
t: 79
t: 80
t: 81
t: 82
t: 83
t: 84
t: 85
t: 86
t: 87
t: 88
t: 89
t: 90
t: 91
t: 92
t: 93
t: 94
t: 95
t: 96
t: 97
t: 98
t: 99
t: 100
t: 101
t: 102
t: 103
t: 104
t: 105
t: 106
t: 107
t: 108
t: 109
t: 110
t: 111
t: 112
t: 113
t: 114
t: 115
t: 116
t: 117
t: 118
t: 119
t: 120
t: 121
t: 122
t: 123
t: 124
t: 125
t: 126
t: 127
t: 128
t: 129
t: 130
t: 131
t: 132
t: 133
t: 134
t: 135
t: 136
t: 137
t: 138
t: 139
t: 140
t: 141
t: 142
t: 143
t: 144
t: 145
t: 146
t: 147
t: 148
t: 149
t: 150
t: 151
t: 152
t: 153
t: 154
t: 155
t: 156
t: 157
t: 1